### Chatbot with message history

In [1]:
# Load env data
import os
from dotenv import load_dotenv
load_dotenv()

# Access Groq Api Key
groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
# Create model
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x767da180f080>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x767da180fbf0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Dayal Arya. I am working as Senior Software Engineer in ABC organization.")])

AIMessage(content="Hi Dayal Arya!\n\nIt's nice to meet you. Thanks for introducing yourself. \n\nBeing a Senior Software Engineer at ABC organization is a great accomplishment. What kind of projects are you working on these days? \n\nI'm always interested in learning more about what people in tech are up to.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 29, 'total_tokens': 96, 'completion_time': 0.121818182, 'prompt_time': 0.00013588, 'queue_time': 0.018769857999999997, 'total_time': 0.121954062}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e0aa850-ddaa-422a-9200-75f28bee8691-0', usage_metadata={'input_tokens': 29, 'output_tokens': 67, 'total_tokens': 96})

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Dayal Arya. I am working as Senior Software Engineer in ABC organization."),
        AIMessage(content="Hello Dayal Arya, it's nice to meet you!\n\nThat's great to hear you're a Senior Software Engineer at ABC organization"),
        HumanMessage(content="What is my name and what did I do?")
    ]
)

AIMessage(content='You are Dayal Arya, and you are a Senior Software Engineer at ABC organization. \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 79, 'total_tokens': 111, 'completion_time': 0.058181818, 'prompt_time': 0.002266144, 'queue_time': 0.021996504, 'total_time': 0.060447962}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f935b5cb-972d-4ff4-9534-a51317705d42-0', usage_metadata={'input_tokens': 79, 'output_tokens': 32, 'total_tokens': 111})

### Message History

In [15]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}

def get_message_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_message_history)

In [16]:
# Create config
config={"configurable": {"session_id": "abc123"}}

In [17]:
with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Dayal Arya. I am working as Senior Software Engineer in ABC organization.")],
    config = config
)

AIMessage(content="Hi Dayal Arya,\n\nIt's nice to meet you! Thanks for introducing yourself.\n\nBeing a Senior Software Engineer at ABC organization sounds very interesting. What kind of projects are you working on? \n\nI'm always eager to learn about different experiences in the software engineering field.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 29, 'total_tokens': 91, 'completion_time': 0.112727273, 'prompt_time': 0.00014872, 'queue_time': 0.02091146, 'total_time': 0.112875993}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7d622687-9848-44a3-b811-162aa47ca420-0', usage_metadata={'input_tokens': 29, 'output_tokens': 62, 'total_tokens': 91})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)
response.content

"Your name is Dayal Arya.  \n\nI'm ready for another question whenever you are!  😊 \n"

In [ ]:
# Change the config
config1 = {"configurable": {"session_id": "abc123"}}
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)

response.content

"Your name is Dayal Arya.  \n\nI'm designed to be helpful and remember information from our conversations.  \n\nIs there anything else I can assist you with? Perhaps you'd like to discuss your work as a Senior Software Engineer at ABC organization?  I'm always interested in learning more about different professions. 😊 \n"

In [24]:
# Change the config -> session_id
config1 = {"configurable": {"session_id": "chat1"}}
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to remember it for our current conversation.\n"

### Prompt Template

In [28]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all of the question best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [29]:
chain.invoke({"messages":[HumanMessage(content="My name is Dayal Arya")]})

AIMessage(content="Hello Dayal Arya, it's nice to meet you!  \n\nI'm ready to assist you with any questions you have. Just ask away, and I'll do my best to provide helpful and informative answers.  \n\nWhat can I help you with today? 🙂 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 30, 'total_tokens': 92, 'completion_time': 0.112727273, 'prompt_time': 0.00014062, 'queue_time': 0.020198108, 'total_time': 0.112867893}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3ce89da0-6b7c-4410-9c08-69965c026c9c-0', usage_metadata={'input_tokens': 30, 'output_tokens': 62, 'total_tokens': 92})

In [30]:
with_message_history = RunnableWithMessageHistory(chain, get_message_history)

In [31]:
config = {"configurable": {"session_id":"chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content="My name is Dayal Arya")],
    config=config
)
response.content

"Hello Dayal Arya! It's nice to meet you.  \n\nI'm ready to help with any questions you have. Just ask away! I'll do my best to provide you with helpful and informative answers. 😊  \n\n"

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)
response.content

'Your name is Dayal Arya.  \n\nI remember that from our first interaction! 😊 \n\nDo you have any other questions for me?\n'

### Prompt Template with Multiple Params

In [34]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpfull assistant. Answer all question to the best of you ability in {language} language"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [37]:
response = chain.invoke(
    {"messages": [HumanMessage("My name is Dayal Arya")], "language": "Hindi"}
)
response.content

'नमस्ते Dayal Arya! \n\nमैं आपकी सहायता करने के लिए यहाँ हूँ।  आप कोई भी प्रश्न पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपको सबसे अच्छी जानकारी हिंदी में प्रदान कर सकूँ।  \n\nआप किस बारे में जानना चाहेंगे? 😊\n\n'

In [38]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_message_history,
    input_messages_key="messages"
)

In [52]:
config = {"configurable": {"session_id":"chat5"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage("My name is Dayal Arya")], "language": "Hindi"},
    config=config
)

response.content

'नमस्ते दयाल आर्य! 😊 \n\nबहुत अच्छा नाम है! \n\nआप मुझसे क्या पूछना चाहते हैं? मैं आपकी पूरी मदद करने के लिए तैयार हूँ।  \n'

In [55]:
response = with_message_history.invoke(
    {"messages": [HumanMessage("What is my name?")], "language": "Hindi"},
    config=config
)

response.content

'आपका नाम दयाल आर्य है।  😊 \n'

### Managing the Conversation History

In [60]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi! I'm bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="Nice!"),
    HumanMessage(content="What's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No problem!"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's 2 + 2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [61]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage("What ice cream do i like?")],
        "language": "English"
    }
)
response.content

'You said you like vanilla ice cream! 😊  \n'

In [62]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage("What math problem did i asked?")],
        "language": "English"
    }
)
response.content

'You asked: "What\'s 2 + 2" \n\n\n\nLet me know if you have any other questions! 😊\n'

In [63]:
# Lets wrap this with message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_message_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id":"chat7"}}

In [64]:
response = with_message_history.invoke(
    {
        
        "messages": messages + [HumanMessage("What ice cream do i like?")],
        "language": "English"
    },
    config=config
)

response.content

'You said you like vanilla ice cream! 😊  \n'